In [3]:
import pandas as pd

In [199]:
# import microbusiness density data
mbd = pd.read_csv("mbd_train.csv")
mbd = mbd.drop(['row_id'], axis=1)

In [200]:
# move county and state to new df which will act as a cfips index

cfips_index = mbd[['cfips', 'county', 'state']].drop_duplicates().reset_index(drop=True)
mbd.drop(['county', 'state'], axis=1, inplace=True)

In [201]:
print(cfips_index.shape)
cfips_index.head()

(3135, 3)


,cfips,county,state
0,1001,Autauga County,Alabama
1,1003,Baldwin County,Alabama
2,1005,Barbour County,Alabama
3,1007,Bibb County,Alabama
4,1009,Blount County,Alabama


In [202]:
print(mbd.shape)
mbd.head()

(122265, 4)


,cfips,first_day_of_month,microbusiness_density,active
0,1001,2019-08-01,3.007682,1249
1,1001,2019-09-01,2.884870,1198
2,1001,2019-10-01,3.055843,1269
3,1001,2019-11-01,2.993233,1243
4,1001,2019-12-01,2.993233,1243


In [203]:
# transpose the date data so each unique cfips is in its own row

df = mbd

# Convert the 'first_day_of_month' column to datetime type
df['first_day_of_month'] = pd.to_datetime(df['first_day_of_month'])

# Pivot the DataFrame
df_pivoted = df.pivot(index='cfips', columns='first_day_of_month', values=['microbusiness_density', 'active'])

# Flatten the column multi-index
df_pivoted.columns = [f'{col[0]}_{col[1].strftime("%Y-%m-%d")}' for col in df_pivoted.columns]

# Reset index to restore 'cfips' as a regular column
df_pivoted.reset_index(inplace=True)

# Merge with the original df
mbd_cfips_grouped = df_pivoted.merge(df['cfips'].drop_duplicates(), on='cfips')


In [204]:
print(mbd_cfips_grouped.shape)
mbd_cfips_grouped.head()

(3135, 79)


,cfips,microbusiness_density_2019-08-01,microbusiness_density_2019-09-01,microbusiness_density_2019-10-01,microbusiness_density_2019-11-01,microbusiness_density_2019-12-01,microbusiness_density_2020-01-01,microbusiness_density_2020-02-01,microbusiness_density_2020-03-01,microbusiness_density_2020-04-01,...,active_2022-01-01,active_2022-02-01,active_2022-03-01,active_2022-04-01,active_2022-05-01,active_2022-06-01,active_2022-07-01,active_2022-08-01,active_2022-09-01,active_2022-10-01
0,1001,3.007682,2.884870,3.055843,2.993233,2.993233,2.969090,2.909326,2.933231,3.000167,...,1401.0,1417.0,1418.0,1433.0,1408.0,1422.0,1461.0,1455.0,1463.0,1472.0
1,1003,7.239156,7.290936,7.425439,7.426071,7.470274,7.413655,7.282522,7.309610,7.568799,...,13247.0,13401.0,13610.0,13668.0,13545.0,14573.0,14686.0,14545.0,14289.0,14320.0
2,1005,1.073138,0.995794,1.160149,1.000628,1.000628,1.027229,1.022314,1.032144,1.086209,...,235.0,239.0,237.0,236.0,235.0,236.0,241.0,237.0,239.0,244.0
3,1007,1.310777,1.305176,1.254761,1.254761,1.265965,1.253638,1.248041,1.264831,1.270428,...,216.0,220.0,225.0,223.0,222.0,227.0,236.0,230.0,234.0,229.0
4,1009,1.544148,1.575892,1.546415,1.573625,1.555485,1.573217,1.536999,1.541526,1.577744,...,778.0,789.0,798.0,783.0,776.0,787.0,813.0,815.0,822.0,813.0


In [207]:
# claculate mbd statistics for each county

df = mbd_cfips_grouped

# Filter columns with "microbusiness_density_" in the label
mbd_columns = [col for col in df.columns if col.startswith('microbusiness_density_')]

# Calculate statistics for filtered columns
statistics = {
    'mbd_mean': df[mbd_columns].mean(axis=1),
    'mbd_min': df[mbd_columns].min(axis=1),
    'mbd_max': df[mbd_columns].max(axis=1),
    'mbd_median': df[mbd_columns].median(axis=1),
    'mbd_variance': df[mbd_columns].var(axis=1),
    'mbd_std': df[mbd_columns].std(axis=1)
}

mbd_stats = pd.DataFrame(statistics)
mbd_stats.set_index(df['cfips'], inplace=True)


In [208]:
mbd_stats

,mbd_mean,mbd_min,mbd_max,mbd_median,mbd_variance,mbd_std
cfips,,,,,,
1001,3.164535,2.884870,3.463856,3.193804,0.026933,0.164113
1003,7.889811,7.239156,8.573463,7.909001,0.133555,0.365452
1005,1.102521,0.982994,1.232074,1.102025,0.005975,0.077298
1007,1.270834,1.186877,1.343183,1.265965,0.001589,0.039863
1009,1.676951,1.536999,1.852060,1.700081,0.009240,0.096125
...,...,...,...,...,...,...
56037,3.119993,2.640386,3.540377,3.163303,0.060228,0.245414
56039,26.337935,25.209639,27.720806,26.090586,0.634601,0.796619
56041,3.946930,3.364125,4.612016,3.954258,0.139415,0.373384


In [209]:
# calculate active stats for each county

df = mbd_cfips_grouped

# Filter columns with "active_" in the label
active_columns = [col for col in df.columns if col.startswith('active_')]

# Calculate statistics for filtered columns
statistics = {
    'mbd_mean': df[active_columns].mean(axis=1),
    'mbd_min': df[active_columns].min(axis=1),
    'mbd_max': df[active_columns].max(axis=1),
    'mbd_median': df[active_columns].median(axis=1),
    'mbd_variance': df[active_columns].var(axis=1),
    'mbd_std': df[active_columns].std(axis=1)
}

active_stats = pd.DataFrame(statistics)
active_stats.set_index(df['cfips'], inplace=True)

In [210]:
active_stats

,mbd_mean,mbd_min,mbd_max,mbd_median,mbd_variance,mbd_std
cfips,,,,,,
1001,1331.717949,1198.0,1472.0,1341.0,6115.313090,78.200467
1003,13064.538462,11464.0,14686.0,13247.0,739530.623482,859.959664
1005,222.102564,200.0,244.0,222.0,172.726046,13.142528
1007,226.769231,212.0,240.0,226.0,50.919028,7.135757
1009,742.230769,679.0,822.0,753.0,1944.024291,44.091091
...,...,...,...,...,...,...
56037,1003.641026,860.0,1132.0,1022.0,5962.130904,77.214836
56039,4953.538462,4750.0,5163.0,4943.0,13520.834008,116.279121
56041,573.666667,488.0,674.0,574.0,3053.175439,55.255547
